In [ ]:
import ROOT
from draw_funcs import draw_chi2_gaussian_sim
from ROOT import TFile
import matplotlib.pyplot as plt

In [ ]:
entry = 6
nevents = 40000
mfield = 1.3
input_path = '/var/kinfit/data/tr_ph_run021142_v5.root'
output_path = 'kinfit_gaussian_fit_etapipi_etato2gamma_run021142_.root'

In [ ]:
ROOT.gInterpreter.ProcessLine('gROOT->LoadMacro("/home/hep/packages/gaussgen/share/gaussgen/rootlogon.C")')

In [ ]:
ROOT.gInterpreter.ProcessLine('gROOT->LoadMacro("/home/hep/packages/kfcmd/share/kfcmd/rootlogon.C")')

In [ ]:
ROOT.gInterpreter.ProcessLine('.L TrPh.C++')

In [ ]:
ROOT.gInterpreter.ProcessLine('TFile fl("{}", "read")'.format(input_path))

In [ ]:
ROOT.gInterpreter.ProcessLine('TrPh a(tr_ph)')

In [ ]:
ROOT.gInterpreter.ProcessLine('a.setNEvents({})'.format(nevents))

In [ ]:
ROOT.gInterpreter.ProcessLine('a.setEntry({})'.format(entry))

In [ ]:
ROOT.gInterpreter.ProcessLine('a.Loop("{output_path}", {mfield})'.format(output_path=output_path, mfield=mfield))

In [ ]:
ROOT.gInterpreter.ProcessLine('fl.Close()')

In [ ]:
fl = TFile.Open(output_path, 'read')

In [ ]:
ROOT.gInterpreter.ProcessLine('''double fcn(double *x, double *p) {
                                        return p[0] * ROOT::Math::chisquared_pdf(x[0], p[1]);}''')

In [ ]:
ROOT.gInterpreter.ProcessLine('TF1 f0("f0", fcn, 0, 30, 2)')

In [ ]:
ROOT.gInterpreter.ProcessLine('f0.SetNpx(10000)')
ROOT.gInterpreter.ProcessLine('f0.SetParameter(0, 1.e+3)')
ROOT.gInterpreter.ProcessLine('f0.SetParameter(1, 10)')

In [ ]:
ROOT.gInterpreter.ProcessLine('kf_chi2->Fit(&f0)')

In [ ]:
draw_chi2_gaussian_sim("kf_chi2", "f0", xlabel=r'$\chi^2$')

In [ ]:
fl.Close()